**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92503 entries, 0 to 92502
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   context      92010 non-null  object
 1   question     92503 non-null  object
 2   statistical  92503 non-null  int64 
 3   theme        92503 non-null  object
 4   file_name    92503 non-null  object
 5   clean_text   92503 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.2+ MB


,context,question,statistical,theme,file_name,clean_text
0,Ondertussen is de eerstelijnszone BruZEL al me...,Zoals alle eerstelijnszones kreeg ook BruZEL h...,0,Brussel en de Vlaamse Rand,1752898.txt,Zoals alle eerstelijnszones kreeg ook BruZEL h...
1,Ondertussen is de eerstelijnszone BruZEL al me...,a)Wat liep er moeilijk?,0,Brussel en de Vlaamse Rand,1752898.txt,Wat liep er moeilijk?
2,Ondertussen is de eerstelijnszone BruZEL al me...,Met welke uitdagingen werd BruZEL het afgelo...,0,Brussel en de Vlaamse Rand,1752898.txt,Met welke uitdagingen werd BruZEL het afgelope...
3,Ondertussen is de eerstelijnszone BruZEL al me...,Hoe kunnen die uitdagingen worden aangepakt?,0,Brussel en de Vlaamse Rand,1752898.txt,Hoe kunnen die uitdagingen worden aangepakt?
4,Ondertussen is de eerstelijnszone BruZEL al me...,b)Wat liep er goed?,0,Brussel en de Vlaamse Rand,1752898.txt,Wat liep er goed?


In [3]:
import re

# Drop unnecessary columns
columns_to_drop = ["context","file_name","question","statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 100].index
df = df[df["theme"].isin(valid_themes)]


# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

In [4]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [5]:
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

theme_counts = train_df["theme_id"].value_counts()



from collections import Counter
print("Class distribution :", Counter(train_labels))


Class distribution : Counter({9: 20094, 2: 11404, 13: 8481, 10: 7612, 4: 6457, 6: 4601, 1: 2595, 8: 2543, 12: 2538, 14: 1588, 3: 1124, 11: 1018, 7: 990, 16: 896, 15: 826, 0: 619, 5: 418, 17: 198})


In [6]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "GroNLP/bert-base-dutch-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts, train_labels, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [7]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# ✅ Clip only the upper bound at 10
clipped_weights = np.clip(class_weights, None, 10.0)

# Convert to PyTorch tensor
class_weights_tensor = torch.tensor(clipped_weights, dtype=torch.float)


In [9]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [10]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,  # 🔥 Warmup ratio of 10% 
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 26/74008 [00:06<4:20:56,  4.73it/s]

 12%|█▎        | 9251/74008 [34:31<3:51:59,  4.65it/s]

{'loss': 2.0657, 'grad_norm': 90.77843475341797, 'learning_rate': 1.9446304442476017e-05, 'epoch': 1.0}


                                                      
 12%|█▎        | 9251/74008 [36:50<3:51:59,  4.65it/s]

{'eval_loss': 1.553849458694458, 'eval_accuracy': 0.6053726825576996, 'eval_precision': 0.6450895435390142, 'eval_recall': 0.6053726825576996, 'eval_f1': 0.6140857039654198, 'eval_runtime': 139.1515, 'eval_samples_per_second': 132.956, 'eval_steps_per_second': 16.622, 'epoch': 1.0}


...

                                                       
100%|██████████| 74008/74008 [4:41:34<00:00,  4.64it/s]

{'eval_loss': 2.3903555870056152, 'eval_accuracy': 0.7556348305497, 'eval_precision': 0.7549339611341016, 'eval_recall': 0.7556348305497, 'eval_f1': 0.7547462129256524, 'eval_runtime': 139.5657, 'eval_samples_per_second': 132.561, 'eval_steps_per_second': 16.573, 'epoch': 8.0}


100%|██████████| 74008/74008 [4:41:38<00:00,  4.38it/s]

{'train_runtime': 16898.7918, 'train_samples_per_second': 35.033, 'train_steps_per_second': 4.379, 'train_loss': 0.7137152676246783, 'epoch': 8.0}


TrainOutput(global_step=74008, training_loss=0.7137152676246783, metrics={'train_runtime': 16898.7918, 'train_samples_per_second': 35.033, 'train_steps_per_second': 4.379, 'total_flos': 1.5578833150446797e+17, 'train_loss': 0.7137152676246783, 'epoch': 8.0})

SAVE MODEL TO KUL DRIVE

In [11]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json


# === Create timestamped save path in OneDrive ===
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
base_path = "C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP"
save_path = os.path.join(base_path, f"Run_{run_id}")
os.makedirs(save_path, exist_ok=True)

# === Save model using Trainer ===
trainer.save_model(save_path)

# === Get predictions ===
preds_output = trainer.predict(test_dataset)
logits = preds_output.predictions
predictions = np.argmax(logits, axis=1)

# === Save predictions to CSV ===
output_df = pd.DataFrame({
    "text": test_texts,  # make sure test_texts is defined
    "true_label": test_labels,  # make sure test_labels is defined
    "predicted_label": predictions,
    "logits": logits.tolist()
})

csv_path = os.path.join(save_path, "test_predictions.csv")
output_df.to_csv(csv_path, index=False)

# === Optional: Save a description file ===
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
geen context meegegeven in zowel train als test
8 epochs 

oversampling via jef zijn weights manier

data set : Grote_data_cleaned
"""

desc_path = os.path.join(save_path, "model_description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(description)

# === Save label mappings ===
mappings_path = os.path.join(save_path, "label_mappings.json")
with open(mappings_path, "w", encoding="utf-8") as f:
    json.dump({
        "theme_to_id": theme_to_id,
        "id_to_theme": {str(k): v for k, v in id_to_theme.items()},  # keys must be str for JSON
        "unique_themes": unique_themes  # Save the unique themes list

    }, f, ensure_ascii=False, indent=4)


print(f"Everything saved in: {save_path}")


100%|██████████| 2313/2313 [02:17<00:00, 16.82it/s]


Everything saved in: C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/GroNLP\Run_2025-04-16_16-30
